In [1]:
import sys
import json
import requests
from base64 import b64encode
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes  
from base64 import b64decode
import re


In [2]:
def findMissing(totalBlock,receivedBlocks):
    missingBlock = None

    for i in range(0,totalBlock):
        blockName = 'block-'+str(i)+'.txt'
        if blockName not in receivedBlocks:
            missingBlock = i
            print("Missing block found : " + blockName)
            
    return missingBlock
            
    

In [3]:
def receiveMissing(missingBlock):
    print("Receiving missing block...")
    url = "http://127.0.0.1:161/getMissing/"+str(missingBlock)
    missingResult =session.get(url)    
    missingResult = json.loads(missingResult.content)
    responseData.append(missingResult)
    print("Missing Block Received..")

In [4]:
#to sort
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):   
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]


In [5]:
def decrpytion(): 
    #sort data
    responseDataSorted = sorted(responseData, key=lambda k: natural_keys(k.get('fileName')), reverse=False)
    print("Decrpytion began ...")
    for currentBlock in responseDataSorted:
        fileName = currentBlock['fileName']
        data = currentBlock['data']
        nonce = nonceList[fileName]
        currentNonce = b64decode(nonce)
        encrpytedBlock = b64decode(data)
        cipher = AES.new(key, AES.MODE_CTR, nonce=currentNonce)    
        decryptedText = cipher.decrypt(encrpytedBlock) 
        decrpytedList.append(decryptedText.decode("utf-8"))
        decrpytedList.append('\n')
        
    print("Decrpytion done..")
    

In [6]:
def exportBook():
    print("Exporting book..")
    decryptionPath = "./clientBlocks/bookDecrypted.txt"
    byteArray = []
    
    file = open(decryptionPath, "wb+")
    #writing encrpytedBlock 
    #converting lines to bytes and inserting them to bytearray
    for currentLine in decrpytedList:
        tempByteArray = bytearray(currentLine, 'utf-8')
        byteArray.append(tempByteArray)

    for currentBytes in byteArray:
          file.write(currentBytes)

            
    file.close()    
    print("Book has been exported..")


In [7]:
def getReceiveBlocks(responseData):
    receivedBlocks = []
    for currentData in responseData:
        receivedBlocks.append(currentData['fileName'])
        
    return receivedBlocks
        

# Main

In [8]:
#Get initial request
session = requests.Session()
request = session.get('http://127.0.0.1:161/')
#convert response to json
response = json.loads(request.content)

responseData = response['data']
key = str.encode(response['key'])
totalBlock = response['totalBlockCount']
nonceList = response['nonceList']
decrpytedList = []

receivedBlocks = getReceiveBlocks(responseData)



missingBlock = findMissing(totalBlock,receivedBlocks)

if missingBlock :
    receiveMissing(missingBlock)

decrpytion()
exportBook()


Missing block found : block-1893.txt
Receiving missing block...
Missing Block Received..
Decrpytion began ...
Decrpytion done..
Exporting book..
Book has been exported..
